## Imported packages here

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor

## Initialised all token keys here in variables

In [6]:
consumer_key='JcKYdqtvb4jQvHuvZueqBGlOU'
consumer_secret='1NcufylSpc2sb83oQWv9qhRGWvYqTFs5ylSoKdPkiJqu07l5ch'
access_token='792341533638946817-shNZlMR3hjQiho72cZwdCb120bwleIR'
access_token_secret='DbKTIhxkymH43wiGV5WmabvPEOWwt9spraQlQnHKUg8j8'

### With the help of tweepy package, the code below is used to to access the twitter API by providing the varaible in which my consumer and access keys data is stored

In [7]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth)

### Got basic details of the MIDAS Labs' twitter account by user name

In [8]:
item = auth_api.get_user('midasIIITD')
print("name: " + item.name)
print("screen_name: " + item.screen_name)
print("description: " + item.description)
print("statuses_count: " + str(item.statuses_count))
print("friends_count: " + str(item.friends_count))
print("followers_count: " + str(item.followers_count))

name: MIDAS IIITD
screen_name: midasIIITD
description: MIDAS is a group of researchers at IIIT-Delhi who study, analyze, and build different multimedia systems for society leveraging multimodal information.
statuses_count: 296
friends_count: 39
followers_count: 174


### Got the entire timeline of the tweets here

In [9]:
tweets = auth_api.user_timeline(screen_name = 'midasIIITD')

### Got the list containing all the statuses of the tweets from the MIDAS twitter timeline

In [37]:
status_list = []
for status in tqdm(Cursor(auth_api.user_timeline, screen_name='midasIIITD', tweet_mode="extended", include_entites = True).items()):
    status_list.append(status)
print("No of statuses ", str(len(status_list)))

296it [00:20, 14.16it/s]


No of statuses  296


### Checking the contents of the ._json to see the contents that it contains

In [38]:
status._json

{'created_at': 'Mon Jul 23 12:53:15 +0000 2018',
 'id': 1021377705084739584,
 'id_str': '1021377705084739584',
 'full_text': 'MIDAS is a group of researchers at IIIT-Delhi who study, analyze, and build different multimedia systems for society leveraging multimodal information. MIDAS stands for Multimodal Digital Media Analysis Lab and it is founded by Dr. Rajiv Ratn Shah. \nURL: https://t.co/sPvbKgXKmg',
 'truncated': False,
 'display_text_range': [0, 278],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/sPvbKgXKmg',
    'expanded_url': 'http://midas.iiitd.edu.in/',
    'display_url': 'midas.iiitd.edu.in',
    'indices': [255, 278]}]},
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1021355762575073281,
  'id_str': '1021

### Creating a list containg only the ._json part of the of thes statuses.

In [39]:
status_json_lst =[]
for status in status_list:
    status_json_lst.append(status._json)

### Dumping all the contents into a .jsonl (JSON Lines) file

In [40]:
import json

with open('midas_tweets.jsonl', 'w') as outfile:
    for entry in status_json_lst:
        json.dump(entry, outfile)
        outfile.write('\n')

### Loading the contents of the .jsonl file into a new list.

In [41]:
tweets = []
for line in open('midas_tweets.jsonl', 'r'):
    tweets.append(json.loads(line))

### Wanted to check where I would find the count of images and this is the code bit which did it...
#### I didn't include the the images in retweeted posts but it could be done to by going into the info about the retweeted post

In [54]:
for tweet in tweets[:20]:
    cnt = 0
    if 'media' in tweet['entities']:
        for image in tweet['entities']['media']:
            cnt+=1
    print(cnt, "\n")

0 

0 

0 

0 

0 

0 

0 

0 

0 

1 

0 

0 

1 

0 

0 

0 

0 

0 

0 

0 



### Created a list of dictionaries to only store the useful info as asked in the task

In [57]:
tweet_dict_lst = []
for tweet in tweets:
    img_cnt = 0
    if 'media' in tweet['entities']:
        for image in tweet['entities']['media']:
            img_cnt+=1
    if img_cnt == 0:
        img_cnt = "None"
    dict_tmp = {'Full Text': tweet['full_text'], 'Date and Time':tweet['created_at'],
                'Number of favorites/likes': tweet['favorite_count'], 
                'Number of retweets': tweet['retweet_count'], 'Number of Images': img_cnt }
    tweet_dict_lst.append(dict_tmp)

### Created a dataframe from the list of dictionaries to present it in a tabular form with the columns in the same order.

In [58]:
tweet_df = pd.DataFrame(tweet_dict_lst)
tweet_df[['Full Text', 'Date and Time','Number of favorites/likes', 'Number of retweets', 'Number of Images' ]]

,Full Text,Date and Time,Number of favorites/likes,Number of retweets,Number of Images
0,@IEEEBigMM19 is also available on Facebook now...,Wed Mar 20 08:19:24 +0000 2019,1,1,None
1,RT @IEEEBigMM19: BigMM 2019 : IEEE BigMM 2019 ...,Wed Mar 20 02:40:07 +0000 2019,0,3,None
2,BigMM 2019 : IEEE BigMM 2019 – Call for Worksh...,Mon Mar 18 02:27:47 +0000 2019,6,3,None
3,"Congratulations @midasIIITD team, Rohan, Prady...",Sun Mar 17 14:22:04 +0000 2019,15,4,None
4,We have emailed the task details to all shortl...,Sat Mar 16 14:06:56 +0000 2019,6,0,None
5,IEEE BigMM 2019 - Call for Workshop Proposals....,Sat Mar 16 09:20:29 +0000 2019,1,1,None
6,"Congratulations! Arijit, Ramit, @debanjanbhucs...",Sat Mar 16 09:14:58 +0000 2019,7,2,None
7,We will be releasing a very interesting task t...,Sat Mar 16 05:13:14 +0000 2019,7,2,None
8,RT @hcdiiitd: Last day to register for #Portfo...,Wed Mar 13 17:09:44 +0000 2019,0,2,None
9,@ACMMM19 @sigmm @TheOfficialACM @acmmmsys @ACM...,Wed Mar 13 04:11:24 +0000 2019,1,0,1
